In [1]:
import json
import time
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from openai import OpenAI
from tqdm import tqdm  # 需要安装: pip install tqdm
import pylitex
import pandas as pd
# ================= 配置区域 =================
API_KEY = 'xxx' 
BASE_URL ='https://api-inference.modelscope.cn/v1'
INPUT_FILE = "./inputs/litex_1106.jsonl"
OUTPUT_FILE = "submit_day1.jsonl"
MODEL_NAME = 'Qwen/Qwen3-Next-80B-A3B-Thinking'
# ===========================================

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

SYSTEM_PROMPT = """You are a Senior Litex Language Engineer. 
Translate mathematical problems into standard Litex code.
Follow the structure strictly: header definitions (if any), claim block, and prove block."""


def get_prompt(question):
    
    LITEX_TEMPLATE = f"""Task: Translate the NL problem into Litex.
Reference Style:
1. For integer division/floor logic, define `fn self_floor(x R) Z`.
2. Structure:
   - `claim:` defines variables, goals, and knowns.
   - `prove:` shows the step-by-step derivation.
   - `=>:` indicates the final answer goal.

Examples:

Example 1:
nl_problem:
In a basketball game, Jon scored 3 points. Jack scored 5 points more than Jon, and Tom scored 4 less than the points of Jon and Jack together. How many points did they score altogether?
Output:
claim:
    forall jon, jack, jon_jack, tom, total N_pos:
        jon = 3
        jack = jon + 5
        jon_jack = jon + jack
        tom = jon_jack - 4
        total = jon + jack + tom
        =>:
            total = 18
    prove:
        jack = 3 + 5
        jack = 8
        jon_jack = 3 + 8
        jon_jack = 11
        tom = 11 - 4
        tom = 7
        total = 3 + 8 + 7
        total = 18

Current Task:
nl_problem: {question}
Output:
"""
    return LITEX_TEMPLATE

In [2]:
def judge_litex_correctness(message):
    """
    judge the code
    """
    result = pylitex.run(message)
    return result["success"], result["message"]

In [3]:
def parse_litex_components(full_code):
    """
    将完整的代码拆解为 header, statement, code 三部分，
    以匹配 submit_day1.jsonl 的格式。
    """
    # 移除 Markdown 标记
    clean_code = full_code.replace("```litex", "").replace("```", "").strip()
    
    # 提取 header (claim: 之前的所有内容)
    header_match = re.split(r'\bclaim:', clean_code, maxsplit=1)
    header = header_match[0] if len(header_match) > 1 else ""
    
    # 提取 formal_statement (从 claim: 开始，到 prove: 之前)
    # 注意：这里我们保留 claim: 关键字
    statement_match = re.search(r'(claim:.*?)(\s+prove:|$)', clean_code, re.DOTALL)
    formal_statement = statement_match.group(1).strip() if statement_match else ""
    
    # 如果没有找到 statement，回退策略
    if not formal_statement and "claim:" in clean_code:
        formal_statement = clean_code
        
    return {
        "header": header,
        "formal_statement": formal_statement,
        "formal_code": clean_code
    }

In [4]:
import time

def generate_single_entry(entry, max_retries=3):
    """
    处理单个条目，包含重试机制。
    如果多次重试均验证失败，强制返回最后一次生成的结果。
    如果无法解析，则字段留空。
    """
    question = entry['nl_problem']
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": get_prompt(question)}
    ]

    generated_code = None  # 初始化变量，用于在循环结束后访问

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
                temperature=0.1
            )
            
            generated_code = response.choices[0].message.content.strip()
            print(f"Attempt {attempt + 1}: Generated Code Received")            
            print(generated_code)
            # 验证
            is_valid, msg = judge_litex_correctness(generated_code)
            
            if is_valid:
                # 成功情况：解析并立即返回
                components = parse_litex_components(generated_code)
                return {
                    "id": entry['id'],
                    "nl_problem": entry['nl_problem'],
                    "formal_type": "Litex",
                    "header": components["header"],
                    "formal_statement": components["formal_statement"],
                    "formal_code": components["formal_code"]
                }
            else:
                # 失败情况：加入错误反馈 (Reflexion) 并继续下一次循环
                feedback = f"Validation failed: {msg}. Please correct the Litex code structure."
                messages.append({"role": "assistant", "content": generated_code})
                messages.append({"role": "user", "content": feedback})
                
        except Exception as e:
            print(f"Error processing {entry['id']} on attempt {attempt}: {e}")
            time.sleep(2)
            
    # --- 兜底逻辑 (Fallthrough Logic) ---
    # 如果代码运行到这里，说明 max_retries 次数用完，且没有一次通过 is_valid check
    
    if generated_code:
        print(f"Max retries reached for {entry['id']}. Returning last invalid result.")
        
        # 尝试解析最后一次生成的代码
        try:
            components = parse_litex_components(generated_code)
        except Exception:
            # 如果解析函数因为代码结构错误而崩溃，则设为空值
            components = {
                "header": None, 
                "formal_statement": None, 
                "formal_code": None
            }

        # 构造最终输出对象（包含空值或部分解析值）
        result = {
            "id": entry['id'],
            "nl_problem": entry['nl_problem'],
            "formal_type": "Litex",
            # 使用 .get 确保安全，如果没有解析出key则为 None
            "header": components.get("header"),
            "formal_statement": components.get("formal_statement"),
            "formal_code": components.get("formal_code"),
            # 建议：可以加一个标记位，方便后续筛选出这些验证失败的数据
            # "review_needed": True 
        }
        return result

    # 如果连一次 API 请求都没成功 (generated_code 依然为 None)，则返回 None
    return None

In [5]:
def read_jsonl_to_df(file_path):
    """
    process the inputs
    """
    data_list = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            try:
                data = json.loads(line)
                data_list.append(data)
            except json.JSONDecodeError as e:
                print(f"警告：第 {line_num} 行解析失败，已跳过：{e}")
    return pd.DataFrame(data_list)

In [6]:
df = read_jsonl_to_df(INPUT_FILE)
result = []
for i in range(len(df)):

    x = df.iloc[i]
    res = generate_single_entry(x)
    print(res)
    result.append(res)
    break

Attempt 1: Generated Code Received
claim:
    forall term1, term2, term3, term4, term5, term6, term7, term8, term9, sum, last_two, tens_digit Z:
        term1 = 11^1 mod 100
        term2 = 11^2 mod 100
        term3 = 11^3 mod 100
        term4 = 11^4 mod 100
        term5 = 11^5 mod 100
        term6 = 11^6 mod 100
        term7 = 11^7 mod 100
        term8 = 11^8 mod 100
        term9 = 11^9 mod 100
        sum = term1 + term2 + term3 + term4 + term5 + term6 + term7 + term8 + term9
        last_two = sum mod 100
        tens_digit = last_two // 10
        =>:
            tens_digit = 5
prove:
    term1 = 11 mod 100
    term1 = 11
    term2 = 11 * 11 mod 100
    term2 = 121 mod 100
    term2 = 21
    term3 = 11 * term2 mod 100
    term3 = 11 * 21 mod 100
    term3 = 231 mod 100
    term3 = 31
    term4 = 11 * term3 mod 100
    term4 = 11 * 31 mod 100
    term4 = 341 mod 100
    term4 = 41
    term5 = 11 * term4 mod 100
    term5 = 11 * 41 mod 100
    term5 = 451 mod 100
    term5 = 5

In [7]:
output_path = f'{outputs_name}.jsonl'
dir_path = os.path.dirname(output_path)
if dir_path:
    os.makedirs(dir_path, exist_ok=True) 

df_processed.to_json(
    output_path,
    orient='records',
    lines=True,
)
print(f'输出结果保存路径为：{output_path}')

NameError: name 'outputs_name' is not defined